## Cell 1: Install Dependencies & Clone Repository

Run this cell first to set up the environment.

In [ ]:
# Install required packages
!pip install -q torch transformers vaderSentiment pandas matplotlib seaborn jupyter kagglehub

# Setup environment variables for Colab
import os
import sys
from pathlib import Path

# Mount Google Drive (optional, for saving outputs)
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    print("✅ Google Drive mounted (optional, for persistent storage)")
except:
    print("⚠️  Google Drive not available (running locally or limited access)")

print("\n" + "=" * 70)
print("📥 Setting up SEC filings dataset...")
print("=" * 70)

# Download SEC EDGAR dataset from Kaggle Hub
try:
    import kagglehub
    
    print("\n🔄 Downloading SEC EDGAR Annual Financial Filings (2021)...")
    print("   Dataset: pranjalverma08/sec-edgar-annual-financial-filings-2021")
    
    # Download dataset
    path = kagglehub.dataset_download("pranjalverma08/sec-edgar-annual-financial-filings-2021")
    print(f"✅ Dataset downloaded to: {path}")
    
    # Setup data directory
    DATA_BASE_DIR = Path(path)
    print(f"\n📂 Scanning dataset structure...")
    
    # List available files
    all_files = list(DATA_BASE_DIR.rglob("*.json"))
    print(f"   Found {len(all_files)} JSON files")
    
    # Create symlink to local data directory for easier access
    LOCAL_DATA_DIR = Path("/content/data")
    LOCAL_DATA_DIR.mkdir(exist_ok=True)
    
    # Copy JSON files to local directory (limited to first 50 for demo)
    import shutil
    sample_files = all_files[:50]
    
    for idx, file_path in enumerate(sample_files, 1):
        try:
            dest = LOCAL_DATA_DIR / file_path.name
            if not dest.exists():
                shutil.copy(str(file_path), str(dest))
        except Exception as e:
            print(f"   ⚠️  Error copying file {file_path.name}: {e}")
    
    print(f"✅ Copied {len(list(LOCAL_DATA_DIR.glob('*.json')))} JSON files to /content/data")
    
except ImportError:
    print("❌ kagglehub not installed, falling back to local data")
except Exception as e:
    print(f"❌ Error downloading from Kaggle Hub: {e}")
    print("   Ensure you have Kaggle credentials configured")
    LOCAL_DATA_DIR = Path("/content/data")
    LOCAL_DATA_DIR.mkdir(exist_ok=True)

# Set working directory and verify structure
os.chdir("/content")
print("\n📂 Current working directory: /content")
print(f"✅ Data directory: /content/data")

# Verify data files
data_files = list(LOCAL_DATA_DIR.glob("*.json"))
if data_files:
    print(f"\n✅ Available data files: {len(data_files)}")
    for i, f in enumerate(data_files[:5], 1):
        size_kb = f.stat().st_size / 1024
        print(f"   {i}. {f.name} ({size_kb:.1f} KB)")
    if len(data_files) > 5:
        print(f"   ... and {len(data_files) - 5} more files")
else:
    print("\n⚠️  No data files found - dataset may not have downloaded correctly")

print("\n" + "=" * 70)


## Cell 2: Setup Environment & Import Modules

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
from typing import Dict, List, Any, Tuple, Optional
import json
import re
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configure paths for Colab
PROJECT_ROOT = Path("/content")
DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = PROJECT_ROOT / "output"
OUTPUT_DIR.mkdir(exist_ok=True)

# Configure visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("=" * 70)
print("Financial Sentiment Analysis - Google Colab Edition")
print(f"Session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")
print("=" * 70)

# ============================================================================
# CORE SENTIMENT ANALYSIS IMPLEMENTATION (from VSCode version)
# ============================================================================

class FinancialSentimentAnalyzer:
    """
    Financial sentiment analyzer using ProsusAI FinBERT model.
    
    FinBERT is a pre-trained NLP model specifically designed for financial text
    sentiment analysis. It provides three-class classification: positive, negative, neutral.
    """
    
    def __init__(self, model_preference: str = "auto", max_length: int = 512):
        """Initialize the sentiment analyzer with FinBERT or VADER fallback."""
        self.model_preference = model_preference
        self.max_length = max_length
        self.use_transformers = False
        self.classifier_pipeline = None
        self.vader_analyzer = None
        
        self._setup_models()
        print(f"✓ Analyzer initialized: {'FinBERT (Transformers)' if self.use_transformers else 'VADER'}")
    
    def _setup_models(self) -> None:
        """Initialize ML models in order of preference."""
        if self.model_preference in ["transformers", "auto"]:
            try:
                from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
                import torch
                
                print("Loading FinBERT model (ProsusAI/finbert)...")
                
                # Determine device
                device = 0 if torch.cuda.is_available() else -1
                device_name = "GPU" if device == 0 else "CPU"
                print(f"Using device: {device_name}")
                
                # Load tokenizer and model
                self.tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
                self.model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
                
                # Create pipeline
                self.classifier_pipeline = pipeline(
                    "text-classification",
                    model=self.model,
                    tokenizer=self.tokenizer,
                    device=device,
                    return_all_scores=True,
                    truncation=True,
                    max_length=self.max_length,
                )
                
                self.use_transformers = True
                print("✓ FinBERT model loaded successfully")
                return
                
            except ImportError as e:
                print(f"⚠ Transformers library not available: {e}. Falling back to VADER.")
            except Exception as e:
                print(f"⚠ Failed to load FinBERT: {e}. Falling back to VADER.")
        
        if self.model_preference in ["vader", "auto"]:
            try:
                from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
                self.vader_analyzer = SentimentIntensityAnalyzer()
                print("✓ VADER sentiment analyzer loaded")
                return
            except ImportError as e:
                print(f"❌ VADER library not available: {e}")
            except Exception as e:
                print(f"❌ Failed to load VADER: {e}")
        
        raise RuntimeError("No sentiment analysis backend available. Install: pip install transformers torch")
    
    def split_into_sentences(self, text: str) -> List[str]:
        """Split text into sentences using pattern matching."""
        if not text.strip():
            return []
        
        sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])', text.strip())
        sentences = [s.strip() for s in sentences if s.strip()]
        sentences = [s for s in sentences if len(s.split()) >= 3]
        
        return sentences
    
    def chunk_sentences(self, sentences: List[str], max_chars: int = 800) -> List[str]:
        """Group sentences into coherent chunks without exceeding character limit."""
        if not sentences:
            return []
        
        chunks = []
        current_chunk = []
        current_length = 0
        
        for sentence in sentences:
            sentence_length = len(sentence)
            
            if sentence_length > max_chars:
                if current_chunk:
                    chunks.append(" ".join(current_chunk))
                    current_chunk = []
                    current_length = 0
                
                words = sentence.split()
                sub_chunk = []
                sub_length = 0
                
                for word in words:
                    word_length = len(word) + 1
                    if sub_length + word_length > max_chars:
                        if sub_chunk:
                            chunks.append(" ".join(sub_chunk))
                        sub_chunk = [word]
                        sub_length = word_length
                    else:
                        sub_chunk.append(word)
                        sub_length += word_length
                
                if sub_chunk:
                    chunks.append(" ".join(sub_chunk))
                continue
            
            if current_length + sentence_length + 1 > max_chars and current_chunk:
                chunks.append(" ".join(current_chunk))
                current_chunk = [sentence]
                current_length = sentence_length
            else:
                current_chunk.append(sentence)
                current_length += sentence_length + 1
        
        if current_chunk:
            chunks.append(" ".join(current_chunk))
        
        return chunks
    
    def _normalize_finbert_label(self, label: str) -> str:
        """Normalize FinBERT label to standard format."""
        label_upper = label.upper()
        if "POSITIVE" in label_upper or label_upper == "POS":
            return "POSITIVE"
        elif "NEGATIVE" in label_upper or label_upper == "NEG":
            return "NEGATIVE"
        else:
            return "NEUTRAL"
    
    def classify_text(self, text: str) -> Dict[str, Any]:
        """Classify text sentiment using available backend."""
        if not text.strip():
            return {"label": "NEUTRAL", "score": 0.0, "backend": "none"}
        
        # Use FinBERT if available
        if self.use_transformers and self.classifier_pipeline:
            try:
                text_to_analyze = text[:2000]
                results = self.classifier_pipeline(text_to_analyze)
                
                if isinstance(results, list) and len(results) > 0:
                    if isinstance(results[0], list):
                        scores = {item['label']: item['score'] for item in results[0]}
                    else:
                        scores = {item['label']: item['score'] for item in results}
                    
                    best_label = max(scores.items(), key=lambda x: x[1])
                    normalized_label = self._normalize_finbert_label(best_label[0])
                    
                    return {
                        "label": normalized_label,
                        "score": best_label[1],
                        "backend": "finbert",
                        "all_scores": scores
                    }
                    
            except Exception as e:
                print(f"⚠ FinBERT analysis failed: {e}. Using VADER.")
        
        # Fallback to VADER
        if self.vader_analyzer:
            vs = self.vader_analyzer.polarity_scores(text)
            comp = vs.get("compound", 0.0)
            
            if comp >= 0.05:
                label = "POSITIVE"
            elif comp <= -0.05:
                label = "NEGATIVE"
            else:
                label = "NEUTRAL"
            
            return {
                "label": label,
                "score": abs(comp),
                "backend": "vader",
                "vader_scores": vs
            }
        
        return {"label": "ERROR", "score": 0.0, "backend": "none"}
    
    def analyze_text(self, text: str, max_chars: int = 800) -> List[Tuple[int, str, Dict]]:
        """Comprehensive text analysis with chunking and sentiment classification."""
        if not text:
            return []
        
        text = re.sub(r'\s+', ' ', text).strip()
        
        paragraphs = [p.strip() for p in re.split(r'\n{2,}', text) if p.strip()]
        if not paragraphs:
            paragraphs = [text]
        
        all_chunks = []
        for paragraph in paragraphs:
            sentences = self.split_into_sentences(paragraph)
            chunks = self.chunk_sentences(sentences, max_chars)
            all_chunks.extend(chunks)
        
        results = []
        for idx, chunk in enumerate(all_chunks, 1):
            try:
                sentiment_result = self.classify_text(chunk)
                results.append((idx, chunk, sentiment_result))
            except Exception as e:
                results.append((idx, chunk, {"label": "ERROR", "score": 0.0, "backend": "none"}))
        
        return results
    
    def aggregate_sentiment(self, results: List[Tuple[int, str, Dict]]) -> Dict[str, Any]:
        """Aggregate sentiment scores from all chunks to determine overall document sentiment."""
        if not results:
            return {
                "label": "NEUTRAL",
                "score": 0.0,
                "confidence": 0.0,
                "compound_score": 0.0,
                "chunk_count": 0,
                "sentiment_distribution": {"Positive": 0, "Negative": 0, "Neutral": 0},
                "positive_ratio": 0.0,
                "negative_ratio": 0.0,
            }
        
        total_weighted = 0.0
        total_score = 0.0
        count = 0
        sentiment_counts = {"Positive": 0, "Negative": 0, "Neutral": 0, "ERROR": 0}
        
        for _, _, res in results:
            label = str(res.get("label", "")).upper()
            score = float(res.get("score", 0.0) or 0.0)
            
            if label.startswith("POS"):
                sign = 1.0
                sentiment_counts["Positive"] += 1
            elif label.startswith("NEG"):
                sign = -1.0
                sentiment_counts["Negative"] += 1
            elif label == "ERROR":
                sign = 0.0
                sentiment_counts["ERROR"] += 1
            else:
                sign = 0.0
                sentiment_counts["Neutral"] += 1
            
            weight = score
            total_weighted += sign * weight
            total_score += score
            count += 1
        
        if count == 0:
            return {
                "label": "NEUTRAL",
                "score": 0.0,
                "confidence": 0.0,
                "compound_score": 0.0,
                "chunk_count": 0,
                "sentiment_distribution": sentiment_counts,
                "positive_ratio": 0.0,
                "negative_ratio": 0.0,
            }
        
        doc_compound = total_weighted / count
        overall_score = total_score / count
        
        positive_ratio = sentiment_counts["Positive"] / count
        negative_ratio = sentiment_counts["Negative"] / count
        
        if positive_ratio > 0.5:
            doc_label = "Positive"
        elif negative_ratio > 0.5:
            doc_label = "Negative"
        elif doc_compound >= 0.05:
            doc_label = "Positive"
        elif doc_compound <= -0.05:
            doc_label = "Negative"
        elif positive_ratio > negative_ratio and positive_ratio > 0.3:
            doc_label = "Positive"
        elif negative_ratio > positive_ratio and negative_ratio > 0.3:
            doc_label = "Negative"
        else:
            doc_label = "Neutral"
        
        confidence = min(overall_score * 1.5, 1.0)
        
        return {
            "label": doc_label,
            "score": abs(doc_compound),
            "confidence": confidence,
            "compound_score": doc_compound,
            "chunk_count": count,
            "sentiment_distribution": sentiment_counts,
            "positive_ratio": positive_ratio,
            "negative_ratio": negative_ratio,
        }


def load_filing_data(file_path: str, sections: Optional[List[str]] = None) -> Dict[str, str]:
    """Load and parse SEC filing JSON data."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        if sections is None:
            sections_data = {
                k: v for k, v in data.items()
                if isinstance(v, str) and len(v.strip()) > 100
            }
        else:
            sections_data = {}
            for section in sections:
                content = data.get(section, "")
                if content and len(content.strip()) > 100:
                    sections_data[section] = content
        
        return sections_data
        
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        return {}
    except json.JSONDecodeError as e:
        print(f"❌ Invalid JSON in {file_path}: {e}")
        return {}
    except Exception as e:
        print(f"❌ Failed to load filing data: {e}")
        return {}


print("✅ Environment ready!")
print("=" * 70)


## Cell 3: Load Sample SEC Filing

In [ ]:
# Find and load first available SEC filing
available_samples = list(DATA_DIR.glob("*.json"))

if available_samples:
    SAMPLE_FILE = str(available_samples[0])
    print(f"✓ Using sample file: {available_samples[0].name}")
else:
    print(f"⚠️  Warning: No sample files found in {DATA_DIR}")
    SAMPLE_FILE = None

SECTIONS_OF_INTEREST = ['item_7', 'item_1A']

if SAMPLE_FILE:
    try:
        filing_data = load_filing_data(SAMPLE_FILE, SECTIONS_OF_INTEREST)
        print("=" * 70)
        print(f"✓ Loaded filing data with {len(filing_data)} sections")
        print(f"  Available sections: {', '.join(filing_data.keys())}")
        print("=" * 70)
        
        # Display section sizes
        for section, text in filing_data.items():
            print(f"  {section}: {len(text):,} characters")
    except FileNotFoundError:
        print(f"⚠️  File not found: {SAMPLE_FILE}")
        filing_data = {}
else:
    filing_data = {}
    print("⚠️  Could not load sample data")


## Cell 4: Initialize FinBERT Analyzer

In [ ]:
# Initialize the sentiment analyzer (FinBERT model)
print("=" * 70)
print("\n🔧 Initializing Sentiment Analyzer...")
print("(First run will download FinBERT model ~500MB)")
print()

analyzer = FinancialSentimentAnalyzer()

print("✅ Ready to analyze filings")
print("=" * 70)

## Cell 5: Analyze Sentiment Per Section

**Key Question**: Is the tone consistent across sections?

In [ ]:
if filing_data:
    print("=" * 70)
    print("\n📊 SENTIMENT COMPARISON - Cross-Section Analysis")
    print("=" * 70)
    comparison_data = []
    
    for section_name, section_text in filing_data.items():
        print(f"\n  Analyzing {section_name}...")
        section_results = analyzer.analyze_text(section_text)
        section_agg = analyzer.aggregate_sentiment(section_results)
        
        comparison_data.append({
            'Section': section_name.replace('item_', 'Item ').upper(),
            'Sentiment': section_agg['label'],
            'Score': f"{section_agg['score']:.3f}",
            'Confidence': f"{section_agg['confidence']:.3f}",
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    print("\n" + comparison_df.to_string(index=False))
    print("\n" + "=" * 70)
    
    # Check for inconsistencies
    sentiments = [d['Sentiment'] for d in comparison_data]
    if len(set(sentiments)) > 1:
        print(f"⚠️  ALERT: Sentiment differs across sections - potential inconsistency detected!")
    else:
        print(f"✅ Consistent tone: All sections show {sentiments[0]} sentiment")
    print("=" * 70)
else:
    print("⚠️  No filing data available")

## Cell 6: Visualize Sentiment Scores

In [ ]:
if 'comparison_df' in locals() and len(comparison_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Convert Score and Confidence to float for plotting
    comparison_df['Score_float'] = comparison_df['Score'].astype(float)
    comparison_df['Confidence_float'] = comparison_df['Confidence'].astype(float)
    
    # Sentiment scores
    colors = ['#2ecc71' if x > 0 else '#e74c3c' if x < 0 else '#95a5a6' 
              for x in comparison_df['Score_float']]
    axes[0].bar(comparison_df['Section'], comparison_df['Score_float'], color=colors)
    axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    axes[0].set_title('Sentiment Score by Section', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Score (Positive → Negative)')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Confidence scores
    axes[1].bar(comparison_df['Section'], comparison_df['Confidence_float'], color='#3498db')
    axes[1].set_title('Analysis Confidence by Section', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Confidence Score')
    axes[1].set_ylim([0, 1.0])
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  Insufficient data for visualization")

## Cell 7: Risk Assessment

**Key Question**: Are there red flags or risk indicators?

In [ ]:
def assess_business_risks(sentiment_results: Dict[str, Any]) -> Dict[str, Any]:
    """
    Assess financial risks from business perspective.
    
    Returns: Dictionary with risk level, specific risks, and recommendations
    """
    risks = []
    severity = "LOW"
    
    positive_ratio = sentiment_results.get('positive_ratio', 0)
    negative_ratio = sentiment_results.get('negative_ratio', 0)
    confidence = sentiment_results.get('confidence', 0)
    
    # Risk 1: Over-optimism (potential misleading statements)
    if positive_ratio > 0.70:
        risks.append("🔴 OVER-OPTIMISM: Unusually positive tone may mask underlying issues")
        severity = "HIGH"
    
    # Risk 2: Distress signals
    if negative_ratio > 0.60:
        risks.append("🔴 DISTRESS SIGNALS: High negative sentiment may indicate financial problems")
        severity = "HIGH"
    
    # Risk 3: Low confidence (vague/unclear disclosures)
    if confidence < 0.30:
        risks.append("🟠 VAGUE LANGUAGE: Low confidence suggests unclear or inconsistent disclosures")
        severity = "MEDIUM" if severity == "LOW" else severity
    
    # Risk 4: Mixed messaging
    if 0.35 < positive_ratio < 0.65 and 0.35 < negative_ratio < 0.65:
        risks.append("🟡 MIXED SIGNALS: Conflicting narratives - company is hedging statements")
        severity = "MEDIUM" if severity == "LOW" else severity
    
    return {
        'severity': severity,
        'risks': risks if risks else ["✅ LOW RISK: Consistent, clear, moderate tone"],
        'confidence': confidence
    }

# Analyze all sections for risks
if filing_data:
    print("\n🚨 RISK ASSESSMENT - Executive Summary")
    print("=" * 70)
    
    all_risks = []
    for section_name, section_text in filing_data.items():
        section_results = analyzer.analyze_text(section_text)
        section_agg = analyzer.aggregate_sentiment(section_results)
        section_risks = assess_business_risks(section_agg)
        
        print(f"\n📌 {section_name.replace('item_', 'Item ').upper()}")
        print(f"   Risk Level: {section_risks['severity']} | Confidence: {section_risks['confidence']:.1%}")
        for risk in section_risks['risks']:
            print(f"   {risk}")
        all_risks.append(section_risks)
    
    # Overall assessment
    print("\n" + "=" * 70)
    print("🎯 OVERALL ASSESSMENT")
    overall_severity = max([r['severity'] for r in all_risks], key=lambda x: {'LOW': 0, 'MEDIUM': 1, 'HIGH': 2}[x])
    print(f"Risk Level: {overall_severity}")
    
    if overall_severity == "HIGH":
        print("⚠️  RECOMMENDATION: Conduct thorough audit of disclosure statements")
    elif overall_severity == "MEDIUM":
        print("⚠️  RECOMMENDATION: Review specific sections flagged above")
    else:
        print("✅ RECOMMENDATION: Standard review procedures sufficient")
else:
    print("⚠️  No filing data available")

## Cell 8: Detailed MD&A Analysis

In [ ]:
if filing_data and 'item_7' in filing_data:
    print("\n📋 DETAILED MD&A ANALYSIS (Management Discussion & Analysis)")
    print("=" * 70)
    
    mdna_text = filing_data['item_7']
    print("\n  Analyzing MD&A section...")
    mdna_results = analyzer.analyze_text(mdna_text)
    mdna_aggregate = analyzer.aggregate_sentiment(mdna_results)
    
    print(f"\nText Length: {len(mdna_text):,} characters")
    print(f"Chunks Analyzed: {len(mdna_results)}")
    
    print(f"\n📊 Sentiment Breakdown:")
    distribution = mdna_aggregate['sentiment_distribution']
    total = sum(distribution.values())
    for sentiment, count in sorted(distribution.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / total * 100) if total > 0 else 0
        print(f"   {sentiment:12s}: {count:3d} chunks ({percentage:5.1f}%)")
    
    print(f"\n🎯 Overall Assessment:")
    print(f"   Sentiment: {mdna_aggregate['label']}")
    print(f"   Score: {mdna_aggregate['score']:.3f} (positive=1.0, negative=-1.0)")
    print(f"   Confidence: {mdna_aggregate['confidence']:.1%}")
    
    # Save for downstream analysis
    mdna_analysis = {
        'results': mdna_results,
        'aggregate': mdna_aggregate
    }
else:
    print("⚠️  MD&A section not available")

## Cell 9: Business Actionable Insights

In [ ]:
print("\n📌 KEY BUSINESS INSIGHTS")
print("=" * 70)

insights = [
    ("AUDITORS", [
        "Use sentiment inconsistency detection to identify sections requiring deeper audit",
        "Flag unusual optimism in problematic business segments",
        "Verify claims in sections with vague or conflicting language"
    ]),
    ("INVESTORS", [
        "Compare sentiment tone to financial metrics (income, cash flow) for coherence",
        "Watch for over-optimism relative to risk factor disclosures",
        "Identify companies hedging negative news with cautious language"
    ]),
    ("REGULATORS", [
        "Monitor for systematically misleading narratives across filings",
        "Compare sentiment trends year-over-year for consistency",
        "Cross-check narrative claims against quantitative financial data"
    ])
]

for stakeholder, use_cases in insights:
    print(f"\n👥 {stakeholder}")
    for i, case in enumerate(use_cases, 1):
        print(f"   {i}. {case}")

## Cell 10: Batch Processing (Multiple Companies)

In [ ]:
print("\n🚀 BATCH PROCESSING - Multiple Companies")
print("=" * 70)

# Demo: First show single filing analysis (already done above)
print("\n📌 SINGLE FILING (Already Analyzed)")
print("-" * 70)
if SAMPLE_FILE:
    print(f"File: {SAMPLE_FILE}")
    if 'overall_severity' in locals():
        print(f"Overall Risk Level: {overall_severity}")
    print("✅ Use this when auditing or analyzing a specific company\n")

# Demo: Now show batch processing capability
data_dir = DATA_DIR
output_dir = OUTPUT_DIR
available_files = list(data_dir.glob("*.json"))

if len(available_files) > 1:
    print(f"📌 BATCH PROCESSING ({len(available_files)} companies found)")
    print("-" * 70)
    print("Processing all filings in data/ directory...\n")
    
    batch_results = []
    
    for file_idx, file_path in enumerate(available_files[:20], 1):  # Limit to 20 for demo
        try:
            print(f"  [{file_idx}/{min(20, len(available_files))}] Processing {file_path.name}...")
            
            # Load filing
            filing_data_batch = load_filing_data(str(file_path), ['item_7', 'item_1A'])
            
            if not filing_data_batch:
                continue
            
            # Extract company ID from filename
            company_id = file_path.stem.split('_')[0] if '_' in file_path.stem else file_path.stem
            
            # Analyze each section
            for section_name, section_text in filing_data_batch.items():
                section_results = analyzer.analyze_text(section_text)
                section_agg = analyzer.aggregate_sentiment(section_results)
                
                batch_results.append({
                    'company_id': company_id,
                    'file_name': file_path.name,
                    'section': section_name,
                    'overall_sentiment': section_agg['label'],
                    'sentiment_score': section_agg['score'],
                    'confidence': section_agg['confidence'],
                    'compound_score': section_agg['compound_score'],
                    'chunk_count': section_agg['chunk_count'],
                    'positive_ratio': section_agg['positive_ratio'],
                    'negative_ratio': section_agg['negative_ratio'],
                })
        
        except Exception as e:
            print(f"  ⚠ Error processing {file_path.name}: {str(e)[:50]}")
            continue
    
    # Create DataFrame and save
    if batch_results:
        batch_df = pd.DataFrame(batch_results)
        
        # Save to CSV
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_file = output_dir / f"batch_analysis_{timestamp}.csv"
        batch_df.to_csv(output_file, index=False)
        
        print("\n✅ Batch Analysis Results:")
        print("-" * 70)
        print(f"\nProcessed {len(batch_df['company_id'].unique())} companies")
        print(f"Analyzed {len(batch_df)} section-level entries")
        print(f"\nResults saved to: {output_file}")
        
        # Display sample results
        print("\nSample Results (first 10 entries):")
        display_cols = ['company_id', 'section', 'overall_sentiment', 'sentiment_score', 'confidence']
        print(batch_df[display_cols].head(10).to_string(index=False))
        
        # Generate comparison visualization
        if len(batch_df) > 0:
            print("\n📊 Generating batch comparison charts...")
            
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
            
            # Chart 1: Sentiment distribution across all companies
            sentiment_counts = batch_df['overall_sentiment'].value_counts()
            colors = {'Positive': '#2ecc71', 'Negative': '#e74c3c', 'Neutral': '#95a5a6'}
            chart_colors = [colors.get(s, '#95a5a6') for s in sentiment_counts.index]
            
            ax1.bar(sentiment_counts.index, sentiment_counts.values, color=chart_colors)
            ax1.set_title('Sentiment Distribution Across All Filings', fontweight='bold')
            ax1.set_ylabel('Number of Sections')
            ax1.set_xlabel('Sentiment')
            
            for i, v in enumerate(sentiment_counts.values):
                ax1.text(i, v + 0.5, str(v), ha='center', va='bottom')
            
            # Chart 2: Average sentiment score by company
            company_avg = batch_df.groupby('company_id')['sentiment_score'].mean().sort_values(ascending=False)
            
            bar_colors = ['#2ecc71' if x > 0.3 else '#e74c3c' if x < -0.3 else '#95a5a6' for x in company_avg.values]
            ax2.barh(range(len(company_avg)), company_avg.values, color=bar_colors)
            ax2.set_yticks(range(len(company_avg)))
            ax2.set_yticklabels(company_avg.index)
            ax2.set_xlabel('Average Sentiment Score')
            ax2.set_title('Average Sentiment by Company', fontweight='bold')
            ax2.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
            
            plt.tight_layout()
            plt.show()
        
        print("\n✅ Use batch processing when:")
        print("   - Analyzing industry trends")
        print("   - Comparing multiple companies")
        print("   - Identifying outliers (unusually optimistic/pessimistic)")
        print("   - Generating regulatory reports")
    else:
        print("⚠️  No valid filings found for batch processing")
    
else:
    print("📌 BATCH PROCESSING")
    print("-" * 70)
    print(f"Only {len(available_files)} file(s) available in data/ directory")
    print("\n✅ Batch processing benefits:")
    print("   - Cross-company sentiment comparison")
    print("   - Industry trend analysis")
    print("   - Risk outlier identification")
    print("   - Exportable results (CSV in output/ folder)")
    print("\n📈 Available files:")
    for i, f in enumerate(available_files[:5], 1):
        print(f"   {i}. {f.name}")
    if len(available_files) > 5:
        print(f"   ... and {len(available_files) - 5} more")

print("\n" + "=" * 70)


## Summary

✅ **Demo Complete!**

This notebook demonstrated:
1. **Single Filing Analysis** - Load and analyze a SEC 10-K filing
2. **Sentiment Comparison** - Compare tone across sections
3. **Risk Assessment** - Flag business risks and inconsistencies
4. **Detailed Analysis** - Deep dive into MD&A section
5. **Stakeholder Insights** - Business value for Auditors, Investors, Regulators
6. **Batch Processing** - Scale analysis to multiple companies

### For Your Assignment:
- **Deliverable 1 (Prototype)**: This notebook + `sentiment_analysis.py` + `batch_analysis.py`
- **Deliverable 2 (Presentation)**: See `Documentations/PRESENTATION_GUIDE.md`

### To Share This Demo:
1. Save this notebook to your Google Colab
2. Click "Share" and set permissions
3. Copy the link and include in your presentation slides
4. During presentation, just click "Run all" cells

---

**HS25 Big Data Assignment - Group Work Part 2**